In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *
import math

### Read in Source Data and Create 3 Scenarios

** Note the Medium scenario utilizes 100 venues which is the main problem to solve **

In [2]:
df = pd.read_csv("./data/TSPTW_150.txt")

In [3]:
all_vals = []
for i in range(df.shape[0]):
    vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
    all_vals.append(vals)

df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
df_data = df_data.iloc[:-1,:]
df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
df_data = df_data.drop(columns=["X", "Y"])

In [4]:
df_small = df_data.iloc[1:].sample(n = 80, replace = False, random_state=42).copy()
df_medium = df_data.iloc[1:].sample(n = 100, replace = False, random_state=42).copy()
df_large = df_data.iloc[1:].sample(n = 120, replace = False, random_state=42).copy()

In [5]:
initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)

In [6]:
df_small = pd.concat([initial, df_small]).reset_index(drop=True)
df_medium = pd.concat([initial, df_medium]).reset_index(drop=True)
df_large = pd.concat([initial, df_large]).reset_index(drop=True)

### Create Distance Matrices

In [7]:
# Compute pairwise times matrix

def distance(df, col, row1, row2):
    c1 = df.loc[row1, col]
    c2 = df.loc[row2, col]
    diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


In [8]:
small_dist = []
for row1 in range(df_small.shape[0]):
    row_vals = []
    for row2 in range(df_small.shape[0]):
        row_vals.append(distance(df_small, "COORDINATES", row1, row2))
    small_dist.append(row_vals)

In [9]:
medium_dist = []
for row1 in range(df_medium.shape[0]):
    row_vals = []
    for row2 in range(df_medium.shape[0]):
        row_vals.append(distance(df_medium, "COORDINATES", row1, row2))
    medium_dist.append(row_vals)

In [10]:
large_dist = []
for row1 in range(df_large.shape[0]):
    row_vals = []
    for row2 in range(df_large.shape[0]):
        row_vals.append(distance(df_large, "COORDINATES", row1, row2))
    large_dist.append(row_vals)

In [11]:
dist_small = np.asarray(small_dist)
dist_medium = np.asarray(medium_dist)
dist_large = np.asarray(large_dist)

### Model Implementation - Small

In [12]:
I = len(dist_small)
J = len(dist_small[0])

# Start Times
A = df_small["READY TIME"].astype(float).to_numpy()

# End Times
B = df_small["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_small

In [13]:
probS = gp.Model("TSPTW - Concerts - Small")

X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-27


In [14]:
for i in range(I):
    # Initiation of Arrival to first node
    probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probS.addConstr(TI[i] >= A[i])

    # end of time window
    probS.addConstr(TJ[i] <= B[i])

    # only one per row
    probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probS.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [15]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [16]:
probS.tune()
for i in range(probS.tuneResultCount):
    print(probS.getTuneResult(i))
probS.optimize()


Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.07s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116  0.00%     -    0s

Explored 1 nodes (161 simplex iterations) i

#### Investigation of MIP Cuts Parameters

In [17]:
result_df_cuts = pd.DataFrame()
result_df_passes = pd.DataFrame()

parameters = [
    "BQPCuts",
    "CliqueCuts",
    "CoverCuts",
    "FlowCoverCuts",
    "FlowPathCuts",
    "GUBCoverCuts",
    "ImpliedCuts",
    "InfProofCuts",
    "LiftProjectCuts",
    "MIPSepCuts",
    "MIRCuts",
    "ModKCuts",
    "NetworkCuts",
    "ProjImpliedCuts",
    "PSDCuts",
    "RelaxLiftCuts",
    "RLTCuts",
    "StrongCGCuts",
    "SubMIPCuts",
    "ZeroHalfCuts"
]

### Model Implementation - Small
I = len(dist_small)
J = len(dist_small[0])

# Start Times
A = df_small["READY TIME"].astype(float).to_numpy()

# End Times
B = df_small["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_small

for parameter in parameters:
    for parakey in [-1,0,1,2]:
        probS = gp.Model("TSPTW - Concerts - Small")

        X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probS.addConstr(TI[i] >= A[i])

            # end of time window
            probS.addConstr(TJ[i] <= B[i])

            # only one per row
            probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probS.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probS.setParam(parameter, parakey)
        probS.optimize()

        params = {
        'Usecase': 'Small',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probS.ObjVal,
        'Runtime' : probS.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
        # Append the dataframe to the result_df
        result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)

parameters = [
    "CutPasses",
    "CutAggPasses",
    "GomoryPasses"
]
for parameter in parameters:
    for parakey in range(100):
        probS = gp.Model("TSPTW - Concerts - Small")

        X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probS.addConstr(TI[i] >= A[i])

            # end of time window
            probS.addConstr(TJ[i] <= B[i])

            # only one per row
            probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probS.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probS.setParam(parameter, parakey)
        probS.optimize()

        params = {
        'Usecase': 'Small',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probS.ObjVal,
        'Runtime' : probS.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
        # Append the dataframe to the result_df
        result_df_passes = result_df_passes.append(params_df, ignore_index=True)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.03s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\2278128562.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.03s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\2278128562.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.03s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

...

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\2278128562.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 98
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.06s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\2278128562.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 99
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.02s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\2278128562.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


### Model Implementation - Medium

In [18]:
I = len(dist_medium)
J = len(dist_medium[0])

# Start Times
A = df_medium["READY TIME"].astype(float).to_numpy()

# End Times
B = df_medium["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_medium

In [19]:
probM = gp.Model("TSPTW - Concerts - Medium")

X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [20]:
for i in range(I):
    # Initiation of Arrival to first node
    probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probM.addConstr(TI[i] >= A[i])

    # end of time window
    probM.addConstr(TJ[i] <= B[i])

    # only one per row
    probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probM.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [21]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [22]:
probM.tune()
for i in range(probM.tuneResultCount):
    print(probM.getTuneResult(i))
probM.optimize()


Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.06s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 simplex iteratio

In [23]:
tot = 0
for v in probM.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj

(0), (52) 1.0 3.605551275463989
(1), (14) 1.0 3.1622776601683795
(2), (66) 1.0 2.8284271247461903
(3), (26) 1.0 8.94427190999916
(4), (50) 1.0 5.830951894845301
(5), (55) 1.0 3.605551275463989
(6), (0) 1.0 27.459060435491963
(7), (87) 1.0 1.0
(8), (77) 1.0 4.242640687119285
(9), (93) 1.0 0.0
(10), (72) 1.0 3.0
(11), (40) 1.0 4.242640687119285
(12), (85) 1.0 8.54400374531753
(13), (41) 1.0 23.194827009486403
(14), (38) 1.0 2.8284271247461903
(15), (83) 1.0 3.1622776601683795
(16), (82) 1.0 6.082762530298219
(17), (7) 1.0 4.123105625617661
(18), (61) 1.0 3.605551275463989
(19), (46) 1.0 1.0
(20), (58) 1.0 12.529964086141668
(21), (62) 1.0 1.4142135623730951
(22), (27) 1.0 4.47213595499958
(23), (39) 1.0 3.605551275463989
(24), (88) 1.0 4.47213595499958
(25), (94) 1.0 4.242640687119285
(26), (25) 1.0 1.4142135623730951
(27), (97) 1.0 4.47213595499958
(28), (11) 1.0 7.211102550927978
(29), (98) 1.0 3.1622776601683795
(30), (45) 1.0 5.656854249492381
(31), (57) 1.0 3.1622776601683795
(32), 

#### Investigation of MIP Cuts Parameters

In [24]:
parameters = [
    "BQPCuts",
    "CliqueCuts",
    "CoverCuts",
    "FlowCoverCuts",
    "FlowPathCuts",
    "GUBCoverCuts",
    "ImpliedCuts",
    "InfProofCuts",
    "LiftProjectCuts",
    "MIPSepCuts",
    "MIRCuts",
    "ModKCuts",
    "NetworkCuts",
    "ProjImpliedCuts",
    "PSDCuts",
    "RelaxLiftCuts",
    "RLTCuts",
    "StrongCGCuts",
    "SubMIPCuts",
    "ZeroHalfCuts"
]

### Model Implementation - Small
I = len(dist_medium)
J = len(dist_medium[0])

# Start Times
A = df_medium["READY TIME"].astype(float).to_numpy()

# End Times
B = df_medium["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_medium

for parameter in parameters:
    for parakey in [-1,0,1,2]:
        probM = gp.Model("TSPTW - Concerts - Small")

        X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probM.addConstr(TI[i] >= A[i])

            # end of time window
            probM.addConstr(TJ[i] <= B[i])

            # only one per row
            probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probM.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probM.setParam(parameter, parakey)
        probM.optimize()

        params = {
        'Usecase': 'Medium',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probM.ObjVal,
        'Runtime' : probM.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
        # Append the dataframe to the result_df
        result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)

parameters = [
    "CutPasses",
    "CutAggPasses",
    "GomoryPasses"
]
for parameter in parameters:
    for parakey in range(100):
        probM = gp.Model("TSPTW - Concerts - Small")

        X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probM.addConstr(TI[i] >= A[i])

            # end of time window
            probM.addConstr(TJ[i] <= B[i])

            # only one per row
            probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probM.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probM.setParam(parameter, parakey)
        probM.optimize()

        params = {
        'Usecase': 'Medium',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probM.ObjVal,
        'Runtime' : probM.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
        # Append the dataframe to the result_df
        result_df_passes = result_df_passes.append(params_df, ignore_index=True)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.07s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\3415274995.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.06s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0     

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\3415274995.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.05s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0     

...

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\3415274995.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 98
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.04s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\3415274995.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 99
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.05s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\3415274995.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


### Model Implementation - Large

In [25]:
I = len(dist_large)
J = len(dist_large[0])

# Start Times
A = df_large["READY TIME"].astype(float).to_numpy()

# End Times
B = df_large["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_large

In [26]:
probL = gp.Model("TSPTW - Concerts - Large")

X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [27]:
for i in range(I):
    # Initiation of Arrival to first node
    probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probL.addConstr(TI[i] >= A[i])

    # end of time window
    probL.addConstr(TJ[i] <= B[i])

    # only one per row
    probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probL.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [28]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [29]:
probL.tune()
for i in range(probL.tuneResultCount):
    print(probL.getTuneResult(i))
probL.optimize()


Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.07s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 simplex iterati

In [30]:
tot = 0
for v in probL.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj

(0), (52) 1.0 3.605551275463989
(1), (38) 1.0 1.4142135623730951
(2), (66) 1.0 2.8284271247461903
(3), (45) 1.0 5.830951894845301
(4), (50) 1.0 5.830951894845301
(5), (55) 1.0 3.605551275463989
(6), (0) 1.0 27.459060435491963
(7), (102) 1.0 0.0
(8), (77) 1.0 4.242640687119285
(9), (93) 1.0 0.0
(10), (72) 1.0 3.0
(11), (40) 1.0 4.242640687119285
(12), (85) 1.0 8.54400374531753
(13), (41) 1.0 23.194827009486403
(14), (1) 1.0 3.1622776601683795
(15), (83) 1.0 3.1622776601683795
(16), (82) 1.0 6.082762530298219
(17), (87) 1.0 5.0990195135927845
(18), (33) 1.0 3.1622776601683795
(19), (46) 1.0 1.0
(20), (12) 1.0 16.97056274847714
(21), (53) 1.0 4.0
(22), (27) 1.0 4.47213595499958
(23), (106) 1.0 5.385164807134504
(24), (88) 1.0 4.47213595499958
(25), (26) 1.0 1.4142135623730951
(26), (25) 1.0 1.4142135623730951
(27), (118) 1.0 3.1622776601683795
(28), (111) 1.0 3.605551275463989
(29), (98) 1.0 3.1622776601683795
(30), (110) 1.0 2.23606797749979
(31), (57) 1.0 3.1622776601683795
(32), (90) 1

#### Investigation of MIP Cuts Parameters

In [31]:
parameters = [
    "BQPCuts",
    "CliqueCuts",
    "CoverCuts",
    "FlowCoverCuts",
    "FlowPathCuts",
    "GUBCoverCuts",
    "ImpliedCuts",
    "InfProofCuts",
    "LiftProjectCuts",
    "MIPSepCuts",
    "MIRCuts",
    "ModKCuts",
    "NetworkCuts",
    "ProjImpliedCuts",
    "PSDCuts",
    "RelaxLiftCuts",
    "RLTCuts",
    "StrongCGCuts",
    "SubMIPCuts",
    "ZeroHalfCuts"
]

### Model Implementation - Small
I = len(dist_large)
J = len(dist_large[0])

# Start Times
A = df_large["READY TIME"].astype(float).to_numpy()

# End Times
B = df_large["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_large

for parameter in parameters:
    for parakey in [-1,0,1,2]:
        probL = gp.Model("TSPTW - Concerts - Large")

        X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probL.addConstr(TI[i] >= A[i])

            # end of time window
            probL.addConstr(TJ[i] <= B[i])

            # only one per row
            probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probL.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probL.setParam(parameter, parakey)
        probL.optimize()

        params = {
        'Usecase': 'Large',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probL.ObjVal,
        'Runtime' : probL.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
        # Append the dataframe to the result_df
        result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)

parameters = [
    "CutPasses",
    "CutAggPasses",
    "GomoryPasses"
]
for parameter in parameters:
    for parakey in range(100):
        probL = gp.Model("TSPTW - Concerts - Small")

        X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probL.addConstr(TI[i] >= A[i])

            # end of time window
            probL.addConstr(TJ[i] <= B[i])

            # only one per row
            probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probL.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probL.setParam(parameter, parakey)
        probL.optimize()

        params = {
        'Usecase': 'Large',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probL.ObjVal,
        'Runtime' : probL.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
        # Append the dataframe to the result_df
        result_df_passes = result_df_passes.append(params_df, ignore_index=True)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.07s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\4209388119.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.07s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\4209388119.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_cuts = result_df_cuts.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.16s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

...

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\4209388119.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 98
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.06s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0    

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\4209388119.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 99
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.06s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0    

C:\Users\tnaga\AppData\Local\Temp\ipykernel_24352\4209388119.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_passes = result_df_passes.append(params_df, ignore_index=True)


In [33]:
result_df_cuts.sort_values(by="Runtime")

,Usecase,Parameter,Parameter Value,Obj Value,Runtime
61,Small,RelaxLiftCuts,0,545.311159,0.039
48,Small,NetworkCuts,-1,545.311159,0.045
54,Small,ProjImpliedCuts,1,545.311159,0.047
63,Small,RelaxLiftCuts,2,545.311159,0.049
58,Small,PSDCuts,1,545.311159,0.051
...,...,...,...,...,...
232,Large,SubMIPCuts,-1,644.067358,0.243
177,Large,FlowPathCuts,0,644.067358,0.243
149,Medium,StrongCGCuts,0,599.776665,0.244
225,Large,RLTCuts,0,644.067358,0.341


In [35]:
result_df_passes.sort_values(by="Runtime")

,Usecase,Parameter,Parameter Value,Obj Value,Runtime
209,Small,GomoryPasses,9,545.311159,0.038
265,Small,GomoryPasses,65,545.311159,0.040
43,Small,CutPasses,43,545.311159,0.041
106,Small,CutAggPasses,6,545.311159,0.041
170,Small,CutAggPasses,70,545.311159,0.041
...,...,...,...,...,...
433,Medium,CutAggPasses,33,599.776665,0.334
740,Large,CutAggPasses,40,644.067358,0.365
735,Large,CutAggPasses,35,644.067358,0.374
646,Large,CutPasses,46,644.067358,0.379


In [36]:
result_df_cuts.to_csv("./Outputs/Cuts_Params.csv")
result_df_passes.to_csv("./Outputs/Passes_Params.csv")